# Linear systems

In [1]:
import numpy as np
from mylinalg import solveLowerTriangular, solveUpperTriangular, lu, lu_solve

# Exercise: LU decomposition

Write a python program to solve it. 
Do not use any linear algebra packackes. 
Use your own linear algebra solvers in `mylinalg.py`.

$$
\boldsymbol{Ax}=
\begin{bmatrix}
2 & 4 & -2 \\
4 & 9 & -3 \\
-2 & -3 & 7 \\
\end{bmatrix}
\cdot
\begin{bmatrix}
x_{1} \\
x_2 \\
x_3 \\
\end{bmatrix}
=
\begin{bmatrix}
2 \\
8 \\
10 \\
\end{bmatrix}
= \boldsymbol{b}
$$

In [2]:
A = [[2,4,-2],[4,9,-3],[-2,-3,7]]

In [3]:
l, u = lu(A)

In [4]:
print(l)
print(u)

[[ 1.  0.  0.]
 [ 2.  1.  0.]
 [-1.  1.  1.]]
[[ 2.  4. -2.]
 [ 0.  1.  1.]
 [ 0.  0.  4.]]


Check $LU = A$

In [5]:
print(np.dot(l,u))
print(A)

[[ 2.  4. -2.]
 [ 4.  9. -3.]
 [-2. -3.  7.]]
[[2, 4, -2], [4, 9, -3], [-2, -3, 7]]


In [6]:
b = np.array([2,8,10])
x = lu_solve(A,b)
print(x)

[-1.  2.  2.]


compare your solution with scipy

In [7]:
from scipy.linalg import lu as scipy_lu
from scipy.linalg import lu_factor as scipy_lu_factor
from scipy.linalg import lu_solve as scipy_lu_solve
from scipy.linalg import solve as scipy_solve

In [8]:
P, L, U = scipy_lu(A)

# A = PLU
print(np.dot(P,np.dot(L,U)))
print(A)

[[ 2.  4. -2.]
 [ 4.  9. -3.]
 [-2. -3.  7.]]
[[2, 4, -2], [4, 9, -3], [-2, -3, 7]]


In [9]:
b = np.array([2,8,10])
lu, piv = scipy_lu_factor(A)
x = scipy_lu_solve((lu, piv), b)
print(x)

[-1.  2.  2.]


In [10]:
x = scipy_solve(A,b)
print(x)

[-1.  2.  2.]


# Apply to the Laplace's equation

Copy your previous codes in `project3_demo1.ipynb` but use your own linear algebra solver.

In [11]:
import numpy as np
from scipy.sparse import dia_array  # if dia_array is not able, use dia_matrix
from scipy.sparse import dia_matrix
from numba import jit, njit, prange
import matplotlib.pyplot as plt
from numba import set_num_threads
nthreads = 8
set_num_threads(nthreads)

In [12]:
# Copy your function from the previous notebook here
def generate_the_laplace_matrix_with_size(N=4):
    """
    assume sqrt(N) is an integer.

    """
    nsq = N*N
    A   = np.zeros((nsq,nsq)) 
 
    id_matrix=-np.identity(N)
    
    zero_matrix=np.zeros((N,N))
    
    ex=np.ones(N)
    data=np.array([-ex,4*ex,-ex])
    offsets=np.array([-1,0,1])
    main_matrix=dia_array((data,offsets),shape=(N,N)).toarray() 
          
    init_matrix_kernel(N,A,zero_matrix,id_matrix,main_matrix)

    return A

@njit(parallel=True)
def init_matrix_kernel(N,A,zero_matrix,id_matrix,main_matrix):
    for i in range(N):
        for j in range(N):
            
            if i==j:
                submatrix=main_matrix
            elif abs(i-j)==1:
                submatrix=id_matrix
            else:
                submatrix=zero_matrix
                
            for ii in range(N):
                for jj in range(N):
                    A[i*N+ii][j*N+jj]=submatrix[ii][jj]
                    
    return A

def generate_the_rhs_vector_with_size(N=4):
    b = np.zeros(N*N)
    b[-N:]=1
    return b

def convert_solution(x):
    usize = np.sqrt(len(x))
    u = x.reshape(int(usize),int(usize)).transpose()
    return u

In [13]:
def solve_laplace(N=16):


    # TODO Copy your solver here
    A = generate_the_laplace_matrix_with_size(N=N)
    b = generate_the_rhs_vector_with_size(N=N)
    #x = linalg.solve(A,b) # use scipy
    x = lu_solve(A,b)      # use our solver
    u = convert_solution(x)

    return u

In [14]:
u = solve_laplace(N=32)

c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\numba\core\typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "..\..\..\AppData\Local\Temp\ipykernel_6472\3301543833.py", line 23:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


KeyboardInterrupt: 

In [ ]:
plt.imshow(u.T,origin="lower")

You could see that our solver is much slower than `scipy.linalg`. Could you speed it up?